In [1]:
import glob
import pybrain
import numpy as np
from pybrain.structure import FeedForwardNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import random
import ast
import time
import sqlalchemy
from sqlalchemy import *
from sqlalchemy import event
import sqlite3
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import *
import urllib2
import urllib
import json
import glob
import pprint
import dateutil.parser
import pprint
import re
from sklearn import linear_model, datasets
import time
from sklearn.naive_bayes import GaussianNB
import nltk
from collections import defaultdict
from sklearn import svm
pp = pprint.PrettyPrinter(indent=4)
import gevent.monkey
gevent.monkey.patch_socket()
import numpy
import time
import sqlalchemy
from sqlalchemy import *
from sqlalchemy import event
from sqlalchemy.dialects.mysql import LONGTEXT
import sqlite3
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import *
import urllib2
import urllib
import json
import pprint
import dateutil.parser
import gevent
import datetime
import marshal
from pylearn2.datasets.dense_design_matrix import DenseDesignMatrix

from __future__ import division
import os

import pylearn2
import numpy as np
from pylearn2.train import Train
from pylearn2.datasets.mnist import MNIST
from pylearn2.models import softmax_regression, mlp, svm
from pylearn2.training_algorithms import bgd, sgd
from pylearn2.termination_criteria import MonitorBased, EpochCounter
from pylearn2.train_extensions import best_params, live_monitoring
from pylearn2.utils import serial
from pylearn2.costs.mlp.dropout import Dropout
from pylearn2.costs.mlp import WeightDecay
from theano import function
from theano import tensor as T
import cPickle as pickle

Using gpu device 0: GRID K520


In [165]:
trainfile = open('usd_eur_trainingdata.csv', 'r')
testfile = open('usd_eur_testingdata.csv', 'r')

X_train = []
Y_train = []

X_test = []
Y_test = []

for line in trainfile:
    line = line.strip().split(',')
    features = line[:-3]
    outputs = line[-3:]
    X_train.append(features)
    Y_train.append(outputs)

for line in testfile:
    line = line.strip().split(',')
    features = line[:-3]
    outputs = line[-3:]
    X_test.append(features)
    Y_test.append(outputs)

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)
    
def load_test_data():
    
    X_out = X_test
    Y_out = Y_test
 
    return DenseDesignMatrix(X=X_out, y=Y_out)
def load_train_data():
    
    X_out = X_train
    Y_out = Y_train
 
    return DenseDesignMatrix(X=X_out, y=Y_out)

In [166]:
print len(X_train)
print len(Y_train)
print len(X_test)
print len(Y_test)

3577
3577
389
389


In [167]:
print np.shape(X_train)
print np.shape(Y_train)

print np.shape(X_test)
print np.shape(Y_test)

(3577, 9)
(3577, 3)
(389, 9)
(389, 3)


In [168]:

train = load_train_data()
valid = load_test_data()
train_monitor = train

In [9]:
monitor = live_monitoring.LiveMonitoring()

In [169]:
if model:
    model.monitor.disable_logging()

monitor.req_sock.close()
monitor.pub_sock.close()
monitor = live_monitoring.LiveMonitoring()

h0 = mlp.Sigmoid(layer_name='h0', dim=9, sparse_init=5)
h1 = mlp.Sigmoid(layer_name='h1', dim=18, sparse_init=5)
ylayer = mlp.Linear(layer_name='y', dim=3, sparse_init=3)

layers = [h0, h1, ylayer]
model = mlp.MLP(layers, nvis=9)

monitoring = dict(valid=valid, train=train_monitor)
termination = EpochCounter(10000)
extensions = [monitor]

algorithm = sgd.SGD(.05, batch_size=30, cost=Dropout(.8),
                     monitoring_dataset = monitoring, termination_criterion = termination)

#algorithm = bgd.BGD(batch_size=2000, cost=WeightDecay({'h0':1., 'h1':1.}),
 #                   monitoring_dataset = monitoring, termination_criterion = termination)

train_job = Train(train, model, algorithm, extensions=extensions)


train_job.main_loop()

Parameter and initial learning rate summary:
	h0_W: 0.0500000007451
	h0_b: 0.0500000007451
	h1_W: 0.0500000007451
	h1_b: 0.0500000007451
	y_W: 0.0500000007451
	y_b: 0.0500000007451
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 0.618457 seconds
Time this epoch: 0.162152 seconds
Time this epoch: 0.157301 seconds
Time this epoch: 0.156941 seconds
Time this epoch: 0.157778 seconds
Time this epoch: 0.156457 seconds
Time this epoch: 0.156536 seconds
Time this epoch: 0.157387 seconds
Time this epoch: 0.157318 seconds
Time this epoch: 0.158328 seconds
Time this epoch: 0.158106 seconds
Time this epoch: 0.157929 seconds
Time this epoch: 0.158115 seconds
Time this epoch: 0.157346 seconds
Time this epoch: 0.157682 seconds
Time this epoch: 0.157339 seconds
Time this epoch: 0.157792 seconds
Time this epoch: 0.157748 seconds
Time this epoch: 0.157934 seconds
Time this epoch: 0.158150 seconds
Time this epoch: 0.157580 seconds
Time this epoch: 0.162263 seconds
Time this epoch: 0.1575

KeyboardInterrupt: 

In [31]:
with open(r"zackmodel2.pickle", "wb") as output_file:
    pickle.dump(model, output_file)

In [144]:
with open(r"zackmodelc.pickle", "rb") as output_file:
    model = pickle.load(output_file)

IOError: [Errno 9] Bad file descriptor

In [173]:
import copy

X = T.matrix()
X = model.get_input_space().make_theano_batch()
Y = model.fprop(X)
f = function([X], Y)

def compare(x, y, set_name, shuffle=False):
    x = copy.deepcopy(x).astype(np.float32)
    y = copy.deepcopy(y).astype(np.float32)
    if shuffle:
        numpy.random.shuffle(y)
    test = f(x)

    comparison = zip(y, test)

    print set_name
    for i in range(1):
        right = 0
        for row in comparison:
            if float(row[0][i]) * float(row[1][i]) > 0:
                right += 1
        print right/float(len(x))
        
compare(X_train, Y_train, "training set")
compare(X_test, Y_test, "test set")
compare(X_train, Y_train,  "training set", True)
compare(X_test, Y_test, "test set", True)

training set
0.485043332401
test set
0.449871465296
training set
0.485043332401
test set
0.449871465296


In [85]:
ben;s
Training set
0.518590998043
0.526139222812
0.51607492312
test set
0.0542353927873
0.0539558289069
0.0567514677104

SyntaxError: invalid syntax (<ipython-input-85-5a7ccd4f90b6>, line 2)

In [ ]:
random
0.477774671512
0.476376852111
0.474326716988

0.519280205656
0.517994858612
0.510711225364

In [154]:
up = 0
down = 0

for x in range(0, 1):
    for row in Y_train:

        if float(row[x]) > 0:
            up += 1
        else:
            down += 1


    print up /(down + up)
    
up = 0
down = 0

for x in range(0, 1):
    for row in Y_test:

        if float(row[x]) > 0:
            up += 1
        else:
            down += 1


    print up /(down + up)

0.5
0.5


In [171]:
def compare(x, y, set_name, shuffle=False):
    x = copy.deepcopy(x).astype(np.float32)
    y = copy.deepcopy(y).astype(np.float32)
    if shuffle:
        numpy.random.shuffle(y)
    test = f(x)

    comparison = zip(y, test)
    numpy.random.shuffle(comparison)
    print set_name
    for i in range(1):
        right = 0
        for row in comparison:
            print row[0][i] , row[1][i]
            if float(row[0][i]) * float(row[1][i]) >= 0:
                right += 1
        print right/float(len(x))
        
compare(X_test, Y_test, "training set")

training set
-0.0001 9.30458e-05
0.0002 9.57008e-05
-0.00025 9.44521e-05
0.00036 9.49848e-05
-0.0007 9.53064e-05
0.0015 9.35847e-05
0.0004 9.59994e-05
-0.00039 9.43916e-05
-0.0003 9.31552e-05
0.0003 9.48308e-05
-0.0007 9.34287e-05
0.0007 9.71084e-05
-1e-05 9.44837e-05
0.002 9.43504e-05
-0.0006 9.47759e-05
0.0008 9.37533e-05
0.00015 9.44906e-05
-0.00034 9.41362e-05
5e-05 9.4405e-05
0.00053 9.49802e-05
-0.0002 9.4362e-05
0.0 9.47029e-05
0.0002 9.47977e-05
0.0001 9.70737e-05
-0.00016 9.4392e-05
0.0004 9.38772e-05
-0.0006 9.40594e-05
0.0009 9.64628e-05
-0.0001 9.43795e-05
-0.0001 9.64154e-05
0.00027 9.32687e-05
-0.0001 9.41828e-05
-0.0015 9.59296e-05
-0.00093 9.40194e-05
-0.0018 9.30449e-05
0.0001 9.49781e-05
0.0 9.59639e-05
-0.0019 9.67756e-05
3e-05 9.48517e-05
0.00068 9.42297e-05
0.00042 9.50412e-05
0.00073 9.4719e-05
0.0005 9.49824e-05
-0.00097 9.50287e-05
-0.0007 9.47157e-05
0.0011 9.49676e-05
-0.0016 9.49315e-05
-0.00052 9.44677e-05
-0.0003 9.58501e-05
-0.0005 9.51499e-05
-0.0011 9.49

In [ ]:
#monitoring = dict(valid=valid)
monitoring = dict()
#termination = MonitorBased(channel_name="valid_y_misclass", N=100)
termination = EpochCounter(10)
#extensions = [best_params.MonitorBasedSaveBest(channel_name="valid_y_misclass",
#save_path="train_best.pkl"), monitor]
extensions = [monitor]


algorithm = sgd.SGD(0.1, batch_size=100, cost=Dropout(),
                    monitoring_dataset = monitoring, termination_criterion = termination)

save_path = "train_bes3t.pkl"
#if os.path.exists(save_path):
#    model = serial.load(save_path)
#else:
#    print 'Running training'
#    train_job = Train(train, model, algorithm, extensions=extensions, save_path="train.pkl", save_freq=1)
#    train_job.main_loop()

train_job = Train(train, model, algorithm, extensions=extensions, save_path="train.pkl", save_freq=1)
train_job.main_loop()


#X = model.get_input_space().make_batch_theano()
#Y = model.fprop(X)

#y = T.argmax(Y, axis=1)
#f = function([X], y)
#yhat = f(test.X)

#y = np.where(test.get_targets())[1]
#print 'accuracy', (y==yhat).sum() / y.size